# script to identify which SNP you can look at

unfortunately this script requires you to get a LDlink API token.
you can get one here and then put it in the script (https://ldlink.nih.gov/?tab=apiaccess).

Then put your SNP into the first cell and run the whole thing.

In [2]:
LDlink_API_token = "faketoken"

SNP_of_interest = "rs4924273"

In [6]:
import pandas as pd
import numpy as np
import requests

In [5]:
SNPs_with_plot = pd.read_csv("http://bartzabel.ls.manchester.ac.uk/orozcolab/SNP2Mechanism/PsA_output_hic_plots/GWAS_corr.txt", sep = "_", header = None)

In [8]:
def get_linked(SNP):
    from io import StringIO
    def ldproxy(variant, population="EUR", r2_threshold=0.8):
        base_url = "https://ldlink.nih.gov/LDlinkRest/ldproxy"
        params = {
            "var": variant,
            "pop": population,
            "r2_threshold": r2_threshold,
            "token":LDlink_API_token,
            "r2_d":"r2",
            "genome_build":"grch38"
        }
        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            data_out = pd.read_csv(StringIO(response.text), sep='\t')
            return data_out
        else:
            raise ValueError(f"Error: {response.status_code}, {response.text}")

    ldproxy_data = ldproxy(SNP, r2_threshold = 0.8)
    ldproxy_data = ldproxy_data[ldproxy_data["R2"] > 0.2]
    linked_snps = ldproxy_data[ldproxy_data["R2"] > 0.8]["RS_Number"].drop_duplicates().to_list()

    return ldproxy_data, linked_snps

In [9]:
ldproxy_data, linked_snps = get_linked(SNP_of_interest)

you can now chose the top SNP from the following table (the one with highest R2 with your SNP)  
if nothing appears then unfortunately this plot was not generated for your SNP

In [12]:
display(ldproxy_data[ldproxy_data["RS_Number"].isin(SNPs_with_plot[2])])

,RS_Number,Coord,Alleles,MAF,Distance,Dprime,R2,Correlated_Alleles,FORGEdb,RegulomeDB,Function
1,rs7170151,chr15:38554477,(C/T),0.2515,-60,1.0000,0.9947,"G=C,C=T",8.0,3a,NaN
4,rs8035957,chr15:38546063,(T/C),0.2515,-8474,0.9947,0.9842,"G=T,C=C",5.0,7,NaN
7,rs36027443,chr15:38554146,(G/A),0.2435,-391,0.9945,0.9428,"G=G,C=A",6.0,4,NaN
9,rs12593201,chr15:38551905,(G/A),0.2406,-2632,1.0000,0.9378,"G=G,C=A",6.0,5,NaN
10,rs28582094,chr15:38551686,(A/G),0.2406,-2851,1.0000,0.9378,"G=A,C=G",6.0,6,NaN
11,rs28536554,chr15:38546231,(A/T),0.2396,-8306,1.0000,0.9327,"G=A,C=T",5.0,6,NaN
15,rs72727388,chr15:38551493,(T/C),0.2505,-3044,0.9469,0.8872,"G=T,C=C",7.0,5,NaN
16,rs72727387,chr15:38551275,(G/A),0.2505,-3262,0.9469,0.8872,"G=G,C=A",7.0,6,NaN
17,rs8043085,chr15:38535939,(G/T),0.2167,-18598,0.9939,0.8090,"G=G,C=T",8.0,5,NaN
